In [ ]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import re
import torch
from keras_preprocessing.sequence import pad_sequences
import time
import datetime

In [ ]:
!pip freeze | grep numpy

numpy==1.21.6


In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 22.9 MB/s 
     |████████████████████████████████| 182 kB 61.8 MB/s 
     |████████████████████████████████| 7.6 MB 50.8 MB/s 


In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Wed Nov 30 12:25:33 2022

@author: MD RAFI
"""
class LocalitySensitiveHashing:
    
    def __init__(self,n_planes,plane_dimensions,n_iterations):
        
        self.n=n_planes
        self.d=plane_dimensions
        self.n_iterations=n_iterations
        self.dic=[]
        self.planes=[]
    def Random_planes(self,n,d):
        
        return np.random.normal(size=(n,d))
    
    
    def fit(self,data):
        
        y_train=data[:,1:]
        #no of iterations
        for i in tqdm(range(self.n_iterations)):
            #generating Random planes
            random=self.Random_planes(self.n, self.d)
            #dot product
            a=np.dot(y_train,random.T)
            #bins
            h=np.where(a>1,1,0)
            #append random planes
            self.planes.append(random)
            #inserting bins into dictionary
            new_dic=defaultdict(list)
            for i in range(len(h)):
              new_dic[str(h[i])].append(data[i,0])
            self.dic.append(new_dic)
    
    def add_point(self,points,index):
       for i in range(len(points)):
           for j in range(len(self.planes)):
                dot=np.dot(points[i],self.planes[j].T)
                dot=np.where(dot>1,1,0)
                self.dic[j][str(dot)].append(index[i,0])
   
    
    def fetch(self,point):
        buckets=set()
        for i in range(len(self.planes)):
          dot=np.dot(point,self.planes[i].T)
          dot=np.where(dot>1,1,0)
          buckets.update(self.dic[i][str(dot)])
          #print(buckets)
        return buckets


    def ShowData(self,vectors,query_vec):
        x=cosine_similarity(vectors[:,1:],query_vec)
        stack=np.array(sorted(np.hstack((vectors[:,0].reshape(-1,1),x.reshape(-1,1))),key=lambda x : x[-1],reverse=True))
        return stack[:,0]


#load tokenizer and model
with open('/content/drive/MyDrive/pickle files/tokenizer.pkl', 'rb') as file:
  tokenizer=pickle.load(file)
with open('/content/drive/MyDrive/pickle files/model.pkl', 'rb') as file:
  bert=pickle.load(file)

## Load data_title Lsh and data_abstract Lsh

with open('/content/drive/MyDrive/pickle files/LSH_title.pkl', 'rb') as file:
  Lsh_data_title=pickle.load(file)

def convert_to_embedding(in_text, MAX_LEN = 150):

    phrase = re.sub(r"http\S+", "", in_text)
    phrase = re.sub(r"won't", "will not",phrase )
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)


    input_ids = tokenizer.encode(
                        phrase, 
                        add_special_tokens = True, 
                        max_length = MAX_LEN,                           
                   )    
    
    #print("input ids",input_ids)

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    #print("results",results)
    
    # Remove the outer list.
    input_ids = results[0]
    #print("input ids",input_ids)

    # Create attention masks    
    attention_mask = [int(i>0) for i in input_ids]
    #print("attention_mask",attention_mask)
    
    # Convert to tensors.
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    #print("input ids",input_ids)
    #print("attention_mask",attention_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)

    #print("input ids",input_ids)
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    bert.eval()

    #input_ids = input_ids.to(device)
    #attention_mask = attention_mask.to(device)
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():        
        o  = bert(
                        input_ids = input_ids, 
                        token_type_ids = None, 
                        attention_mask = attention_mask)
        
        h_s = o[1][1:]

    layer_i = 11 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]

    #print(h_s[11].shape)
    # Extract the embedding.
    embedding = h_s[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    embedding = embedding.detach().cpu().numpy()

    return(embedding)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


with open('/content/drive/MyDrive/pickle files/Final_dataset1.pkl', 'rb') as file:
  dataset=pickle.load(file)

 
data_title_embeddings = pd.read_csv("/content/drive/MyDrive/pickle files/final_embeddings")
data_title_embeddings = data_title_embeddings.drop("Unnamed: 0" , axis=1)





In [ ]:
t0 = time.time()
query="Principal component analysis"
query_embedding=convert_to_embedding(query)
indices_data_title=list(map(int,Lsh_data_title.fetch(query_embedding)))

sorted_title_indices=Lsh_data_title.ShowData(data_title_embeddings.iloc[indices_data_title].to_numpy(),np.array([query_embedding]))
output = dataset.iloc[sorted_title_indices]
print("Time taken :",format_time(time.time() - t0))
out_json = output.to_json(orient='split') 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Time taken : 0:00:01


In [ ]:
out_json

'{"columns":["year","title","abstract","auhtors"],"index":[16372,17462,3172,4641,1597,2939,14608,2167,4937,1581,4087,22403,23599,20779,12902,17448,2740,2260,911,27334,15783,18701,22809,11943,19999,19686,3094,24441,28095,1086,5715,2572,5927,20276,18537,14631,7574,18845,22094,21279,16252,10129,28852,19250,14301,5273,1478,4435,20269,25058,15750,868,17958,9421,2724,12744,26978,20217,21992,23219,29082,1265,10593,13391,1673,17354,26835,17879,28347,26812,20682],"data":[[2017,"Contrastive Principal Component Analysis","We present a new technique called contrastive principal component analysis\\n(cPCA) that is designed to discover low-dimensional structure that is unique to\\na dataset, or enriched in one dataset relative to other data. The technique is\\na generalization of standard PCA, for the setting where multiple datasets are\\navailable -- e.g. a treatment and a control group, or a mixed versus a\\nhomogeneous population -- and the goal is to explore patterns that are specific\\nto one o

In [ ]:
  output

,year,title,abstract,auhtors
16372,2017,Contrastive Principal Component Analysis,We present a new technique called contrastive ...,"Abubakar Abid, Martin J. Zhang, Vivek K. Ba..."
17462,2012,Generalized Principal Component Analysis (GPCA),This paper presents an algebro-geometric solut...,"Rene Vidal, Yi Ma, Shankar Sastry"
3172,2008,Robust Kernel Principal Component Analysis,Kernel Principal Component Analysis (KPCA) is ...,NaN
4641,2013,Similarity Component Analysis,Measuring similarity is crucial to many learni...,NaN
1597,2000,Sparse Kernel Principal Component Analysis,'Kernel' principal component analysis (PCA...,"Charles Isbell , Paul Viola"
...,...,...,...,...
26835,2017,Slim Embedding Layers for Recurrent Neural Lan...,Recurrent neural language models are the state...,"Zhongliang Li, Raymond Kulhanek, Shaojun Wa..."
17879,2017,One Shot Joint Colocalization and Cosegmentation,This paper presents a novel framework in which...,Abhishek Sharma
28347,2013,Dealing with Uncertainty in Situation Assessme...,The situation assessment problem is considered...,"Charles Castel, Corine Cossart, Catherine T..."
26812,2017,Incorporating Syntactic Uncertainty in Neural ...,Incorporating syntactic information in Neural ...,"Poorya Zaremoodi, Gholamreza Haffari"


In [ ]:

def test(text:None):
  print("Executing")
  t0 = time.time()
  output = pd.DataFrame()
  if text == None:
    query_text_test = "HUMAN BRAIN MIMICS"
  else:
    query_text_test = text
  
  query_embedding=convert_to_embedding(query_text_test)
  indices_data_title=list(map(int,Lsh_data_title.fetch(query_embedding)))

  sorted_title_indices=Lsh_data_title.ShowData(data_title_embeddings.iloc[indices_data_title].to_numpy(),np.array([query_embedding]))
  output = dataset.iloc[sorted_title_indices]
  print("Time taken :",format_time(time.time() - t0))
  out_json = output.to_json(orient='split')  
  return(out_json)





array(['Variational inference is a scalable technique for approximate Bayesian inference.  Deriving variational inference algorithms requires tedious model-specific calculations; this makes it difficult for non-experts to use.  We propose an automatic variational inference algorithm, automatic differentiation variational inference (ADVI); we implement it in Stan (code available), a probabilistic programming system.  In ADVI the user provides a Bayesian model and a dataset, nothing else.  We make no conjugacy assumptions and support a broad class of models. The algorithm automatically determines an appropriate variational family and optimizes the variational objective. We compare ADVI to MCMC sampling across hierarchical generalized linear models, nonconjugate matrix factorization, and a mixture model. We train the mixture model on a quarter million images.  With ADVI we can use variational inference on any model we write in Stan.',
       'Behavioral cloning reduces policy learning to 